In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install mne
! pip install pyriemann

     |████████████████████████████████| 7.0MB 2.2MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 


In [ ]:
import zipfile

# Will unzip the files so that you can see them..
with zipfile.ZipFile("/content/drive/MyDrive/2021_VIIT08_P300/Dataset/inria-bci-challenge/test.zip","r") as z:
    z.extractall("./train")

In [ ]:
with zipfile.ZipFile("/content/drive/MyDrive/2021_VIIT08_P300/Dataset/inria-bci-challenge/test.zip","r") as z:
    z.extractall("./test")

In [ ]:
import numpy as np                                      # for dealing with data
from scipy.signal import butter, sosfiltfilt, sosfreqz  # for filtering
import matplotlib.pyplot as plt                         # for plotting
from scipy import interp
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
import os
from os import listdir
from os.path import isfile, join, isdir
import pickle

In [ ]:
# Create filtering variables
fs = 200.0     # 200 Hz sampling rate
lowcut = 1.0   # 0.1 Hz is the lowest frequency we will pass
highcut = 40.0 # 40 Hz is the highest frequency we will pass.

In [ ]:
def butter_bandpass_filter(raw_data, fs, lowcut = 1.0, highcut =40.0, order = 5):
    '''
    The filter I want to apply to my raw eeg data.
    :raw_data (nparray): data you want to process
    :fs (float): sampling rate
    :lowcut (float, optional): lowest frequency we will pass
    :highcut (float, optional): highest frequency we will pass
    :order (int, optional): order of filter
    '''
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog = False, btype = 'band', output = 'sos')
    filted_data = sosfiltfilt(sos, raw_data)
    return filted_data

In [ ]:
epoch_s = 0      # epoch starting time relative to stmulus in miliseconds
epoch_e = 700    # epoch ending time relative to stmulus in miliseconds
bl_s = 0         # baseline starting time relative to stmulus in miliseconds
bl_e = 100       # baseline ending time relative to stmulus in miliseconds


# number of mark per epoch
epoch_len = int((abs(epoch_s) + abs(epoch_e)) * (fs / 1000))

In [ ]:
train_subj_num = 16
test_subj_num = 10
stimulus_per_subj = 340
trial_per_subj = 5

channels = ['Fp1', 'Fp2', 'AF7', 'AF3', 'AF4', 'AF8', 'F7', 'F5', 'F3', 'F1',
    'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz',
    'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2',
    'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4',
    'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8',
    'PO7', 'POz', 'P08', 'O1', 'O2']

In [ ]:
import pandas as pd
train_labels = pd.read_csv("/content/drive/MyDrive/2021_VIIT08_P300/Dataset/inria-bci-challenge/TrainLabels.csv")
#sample_train_data = pd.read_csv("/content/drive/MyDrive/2021_VIIT08_P300/Dataset/inria-bci-challenge/train.zip/Data_S02_Sess01.csv")
sample_train_data = pd.read_csv("./train/Data_S01_Sess01.csv")

In [ ]:
train_labels

,IdFeedBack,Prediction
0,S02_Sess01_FB001,1
1,S02_Sess01_FB002,1
2,S02_Sess01_FB003,0
3,S02_Sess01_FB004,0
4,S02_Sess01_FB005,1
...,...,...
5435,S26_Sess05_FB096,1
5436,S26_Sess05_FB097,0
5437,S26_Sess05_FB098,0
5438,S26_Sess05_FB099,0


In [ ]:

df2 = sample_train_data[sample_train_data['FeedBackEvent'] == 1]

In [ ]:
df2


,Time,Fp1,Fp2,AF7,AF3,AF4,AF8,F7,F5,F3,F1,Fz,F2,F4,F6,F8,FT7,FC5,FC3,FC1,FCz,FC2,FC4,FC6,FT8,T7,C5,C3,C1,Cz,C2,C4,C6,T8,TP7,CP5,CP3,CP1,CPz,CP2,CP4,CP6,TP8,P7,P5,P3,P1,Pz,P2,P4,P6,P8,PO7,POz,P08,O1,O2,EOG,FeedBackEvent
3936,19.680,-865.907719,-336.865723,395.619290,-256.963125,7.128198,169.644889,2.182499,198.410807,248.434759,425.365991,136.510955,-138.103378,24.324664,433.272575,96.433906,271.582105,197.768098,261.949333,375.321843,221.810441,82.159881,312.267221,301.031191,240.498672,400.565037,244.948552,301.137152,131.351555,132.809316,230.872268,-10.026648,167.572375,318.897235,202.787861,284.227967,173.458859,130.223382,89.027509,112.890367,268.878430,615.197831,140.275432,247.175748,-83.461986,193.518058,235.431491,95.638154,199.950368,331.599038,267.292339,238.472690,192.562898,-59.426189,118.574484,83.818863,-215.689171,-329.776806,1
5430,27.150,-844.269907,-332.449251,363.796610,-240.592566,19.527248,165.711598,2.643127,192.153705,234.734831,415.735897,135.786607,-139.470921,30.876938,439.922681,124.087039,264.017285,187.473344,253.130887,353.196313,208.047526,77.607040,307.998541,299.079444,243.526212,362.917036,216.836397,282.806720,113.833311,119.102564,227.224374,0.643498,163.541754,315.579475,185.235776,270.682876,153.104579,112.441793,69.415568,106.567053,263.161908,609.385380,135.543006,217.464165,-114.073980,139.845921,213.414969,63.126737,173.760409,306.825765,253.281648,222.875340,176.315353,-70.718299,103.311274,63.083145,-195.812350,-355.112319,1
6923,34.615,-744.687215,-261.898731,395.128181,-176.330388,89.044518,242.301264,51.207161,249.585624,283.710074,448.918230,186.604281,-85.653214,65.655827,496.584827,186.221783,301.499328,232.472769,299.939973,403.418691,269.520053,123.764444,347.732845,351.339338,281.644801,390.749187,260.545204,315.528041,159.789695,162.035128,262.694414,41.467514,194.360647,343.808636,222.854315,309.015820,198.682635,153.506421,103.315934,134.281604,295.261175,632.516122,169.230279,228.101906,-63.837073,211.803456,245.526524,113.863424,203.274286,351.958010,297.830896,254.633729,213.337791,-29.031159,152.330334,109.413904,-112.257429,-218.617422,1
8420,42.100,-750.132629,-257.870971,343.752429,-188.249109,81.705075,269.817711,17.013881,208.210085,252.241392,413.205210,167.163973,-102.674896,55.568003,469.991890,202.597576,270.567614,211.035179,278.730384,370.403501,235.424347,107.264113,328.973758,343.690676,282.858711,366.120329,245.725914,304.886026,150.510750,146.455437,247.397251,46.028295,194.192246,336.912414,199.088930,292.145122,186.803170,135.491430,92.978364,135.227674,297.077956,629.231981,167.139910,213.911200,-45.218497,240.176911,247.298764,118.120030,165.455204,364.533305,296.874655,268.385869,204.467222,-25.441270,164.913765,102.904626,-55.840202,-208.543986,1
9916,49.580,-729.811446,-223.470216,357.298291,-186.645391,99.255454,257.363895,16.934330,214.782003,230.506949,393.843882,146.744632,-115.072129,45.367713,429.301075,195.335704,242.559701,187.544844,255.301662,337.191334,218.978270,89.218053,310.661424,325.340716,257.858347,334.781637,213.113728,261.165732,120.062001,124.641225,226.612905,26.588772,168.668056,310.304187,175.905798,274.318189,155.767024,117.232439,60.290388,113.754128,267.023748,605.459529,146.733422,193.553786,-74.015259,183.417108,248.817549,87.808346,135.598315,328.584747,275.480140,254.013670,174.710879,-48.521572,137.117964,81.026399,-58.655249,-125.541783,1
12323,61.615,-692.695231,-198.430503,321.850346,-179.707645,52.690514,195.960857,-2.268792,186.902050,178.883390,353.616333,140.596630,-126.556375,23.866958,364.349608,151.205168,239.910430,162.257403,225.428155,302.112338,202.211010,70.936214,287.230815,299.481920,240.457161,302.751477,205.116001,220.830750,99.109025,99.580680,212.906421,18.880592,145.633806,295.103617,155.353937,239.492406,144.457059,95.689833,39.801778,98.963712,243.315645,526.466937,134.587504,178.944309,-97.196849,124.899662,221.086007,66.248204,103.551547,282.231405,244.309493,224.084775,141.505684,-64.724

In [ ]:
train_list_arr = np.array(sorted(listdir('./train')))

train_list_np = np.reshape(train_list_arr,(train_subj_num, trial_per_subj))

test_list_arr = np.array(sorted(listdir('./test')))
test_list_np = np.reshape(test_list_arr[0:50], (test_subj_num, trial_per_subj))
print(train_list_np.shape,test_list_np.shape)

train_data_list = np.empty(
    (0, stimulus_per_subj, len(channels), epoch_len), float)
test_data_list = np.empty(
    (0, stimulus_per_subj, len(channels), epoch_len), float)
print(train_data_list.shape,test_data_list.shape)

ValueError: ignored

In [ ]:
# Load our data
X_train = np.load('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/X_train.npy')
X_test = np.load('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/X_test.npy')
Y_train = np.load('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/Y_train.npy')

# Only used for scoring
Y_test = np.reshape(pd.read_csv('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/inria-bci-challenge/true_labels.csv', header=None).values, 3400)

In [ ]:
train_list_np

NameError: ignored

In [ ]:
test_list_np

In [ ]:
def generate_epoch(file_path, channels, fs, eeg_filter, stimulus_times=None, baseline=True,  epoch_s=0, epoch_e=700, bl_s=0, bl_e=100):
    """
    :description: Generating epoch given csv file. Make sure the csv file layout meets the requirement.
        It should contain 'Time' column that represents timepoints, and the time should start from 0.
        If your csv file does not have FeedBackEvent indicating the stimulus, you must pass stumulus_times.
        Here we used a butter bandpass filter, but you can change to your favorite one.

    :file_path (String): path to your csv file
    :channels ([String]): array of channels to epoch
    :fs (float): sampling rate
    :eeg_filter (function): the filter you want to apply to raw eeg data
    :stimulus_times ([float], optional): The time points that stimulus occur
    :baseline (boolean, optional): whether you want to apply baseline correction after epoching
    :epoch_s (int, optional): epoch starting time relative to stmulus in miliseconds
    :epoch_e (int, optional): epoch ending time relative to stmulus in miliseconds
    :bl_s (int, optional): baseline starting time relative to stmulus in miliseconds
    :bl_e (int, optional): baseline ending time relative to stmulus in miliseconds

    :rtype (3d-nparray): epoched data with dimension (stimulus_per_trial, number_of_channels, number_of_time_points)
    """
    # read data and data selection
    train_data = pd.read_csv(file_path)

    train_data.loc[:, 'Time'] = train_data.loc[:, 'Time']*1000
    raw_eeg = train_data[channels].values.T

    time_df = train_data['Time'].values
    train_data['index'] = train_data.index.values
    if stimulus_times is None:
        mark_indices = np.asarray(
            train_data[train_data['FeedBackEvent'] == 1].index).flatten()
    else:
        mark_indices = np.round(np.asarray(
            stimulus_times).flatten() * fs).astype(int)

    # Define the bounds of our epoch as well as our baseline
    # index in epoch_df where our baseline begins
    b_s = int((abs(epoch_s) + bl_s) * (fs / 1000))
    # index in epoch_df where our baseline ends
    b_e = int((abs(epoch_s) + bl_e) * (fs / 1000))
    # Let's calculate the length our epoch with our given sampling rate
    epoch_len = int((abs(epoch_s) + abs(epoch_e)) * (fs / 1000))

    # Let's define some helpful variables to make our extraction easier
    # effectively the number of indices before marker we want
    e_s = int((epoch_s * (fs / 1000)))
    # effectively the number of indices after marker we want
    e_e = int((epoch_e * (fs / 1000)))

    # Epoch the data
    final_epoch = np.empty((mark_indices.shape[0], epoch_len, 0), float)
    for channel in channels:
        epoch = np.zeros(shape=(int(mark_indices.shape[0]), epoch_len))
        raw_eeg = train_data[channel].values

        ################# You may want to apply your own filter ################
        clean_eeg = eeg_filter(raw_eeg, fs, lowcut, highcut, 5)
        ########################################################################

        for i, mark_idx in enumerate(mark_indices):
            # grab the appropriate samples around the stimulus onset
            epoch[i, :] = clean_eeg[mark_idx + e_s: mark_idx + e_e]

        # Baseline correction
        if baseline:
            for i in range(0, int(epoch.shape[0])):
                epoch[i, :] = epoch[i, :] - np.mean(epoch[i, b_s:b_e])

        # stack epoch of each channel
        final_epoch = np.dstack((final_epoch, epoch))
    final_epoch = np.swapaxes(final_epoch, 1, 2)
    return final_epoch

In [ ]:
if not isfile("./train_data.npy"):
    for training_participant_id in range(train_subj_num):
        subject_dir_list = train_list_np[training_participant_id]
        subject_epoch = np.empty((0, len(channels), epoch_len), float)
        index=1
        for trial_id in range(trial_per_subj):
            subject_dir = subject_dir_list[trial_id]
            data = generate_epoch('./train/'+subject_dir, channels, fs,
                butter_bandpass_filter, epoch_s = epoch_s, epoch_e = epoch_e, bl_s = bl_s, bl_e = bl_e)
            subject_epoch = np.vstack((subject_epoch, data))
        print(subject_epoch.shape)
        subject_epoch = np.reshape(
            subject_epoch, (index, stimulus_per_subj, len(channels), epoch_len))
        train_data_list = np.vstack((train_data_list, subject_epoch))
        index=index+1

    print('Epoched training data shape: ' + str(train_data_list.shape))

    for testing_participant_id in range(test_subj_num):
        subject_dir_list = test_list_np[testing_participant_id]
        subject_epoch = np.empty((0, len(channels), epoch_len), float)
        for trial_id in range(trial_per_subj):
            subject_dir = subject_dir_list[trial_id]
            data = generate_epoch('./test/'+subject_dir, channels, fs,
                butter_bandpass_filter, epoch_s = epoch_s, epoch_e = epoch_e, bl_s = bl_s, bl_e = bl_e)
            subject_epoch = np.vstack((subject_epoch, data))
        subject_epoch = np.reshape(
            subject_epoch, (1, stimulus_per_subj, len(channels), epoch_len))
        test_data_list = np.vstack((test_data_list, subject_epoch))

    print('Epoched testing data shape: ' + str(test_data_list.shape))

    np.save('./train_data.npy', train_data_list)
    np.save('./test_data.npy', test_data_list)

NameError: ignored

In [ ]:
train_data_list = np.load('./train_data.npy')
test_data_list = np.load('./test_data.npy')
print('Epoched training data shape: ' + str(train_data_list.shape))
print('Epoched testing data shape: ' + str(test_data_list.shape))

In [ ]:
pip install -U numpy scipy scikit-learn

In [ ]:
pip install scikit-learn

In [ ]:
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace

In [ ]:
Y_train = pd.read_csv('../input/inria-bci-challenge/TrainLabels.csv')['Prediction'].values

In [ ]:
train_data_list.shape

In [ ]:
print(stimulus_per_subj,len(channels),epoch_len)

In [ ]:
if not isfile('./X_train.npy'):
    XC = XdawnCovariances(nfilter=5) # our transformer
    TS = TangentSpace(metric='riemann')
    # collapse first two dimension
    train_data = np.reshape(train_data_list, 
    (16 * stimulus_per_subj, len(channels), epoch_len))
    test_data = np.reshape(test_data_list, 
    (test_subj_num * stimulus_per_subj, len(channels), epoch_len))

    # transform our data
    X_train = XC.fit_transform(train_data, Y_train)
    
    X_train = TS.fit_transform(X_train)
    X_test = XC.transform(test_data)
    X_test = TS.transform(X_test)

    # save to local
    np.save('./X_train', X_train)
    np.save('./X_test', X_test)
    np.save('./Y_train', Y_train)

In [ ]:
# Load our data
X_train = np.load('./X_train.npy')
X_test = np.load('./X_test.npy')
Y_train = np.load('./Y_train.npy')

# Only used for scoring
Y_test = np.reshape(pd.read_csv('true_labels.csv', header=None).values, 3400)

In [ ]:
print('Transformed training data shape: ' + str(X_train.shape))
print('Training label shape: ' + str(Y_train.shape))
print('Transformed testing data shape: ' + str(X_test.shape))
print('Testing label shape: ' + str(Y_test.shape))

Transformed training data shape: (5440, 210)
Training label shape: (5440,)
Transformed testing data shape: (3400, 210)
Testing label shape: (3400,)


In [ ]:
!pip install catboost

     |████████████████████████████████| 67.3MB 46kB/s 


In [ ]:
import seaborn as sns
import statsmodels.api as sm
from catboost import CatBoostClassifier
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
learning_rate=  np.arange(0.05, 0.31, 0.05),
max_depth=np.arange(5, 16, 1, dtype=int)
colsample_bylevel= np.arange(0.3, 0.8, 0.1)
n_estimators=[50,60,70,80]
eval_metric=     'RMSE'

param={'learning_rate': learning_rate,
       'max_depth': max_depth,
       'colsample_bylevel':colsample_bylevel,
       'n_estimators': n_estimators,
       'eval_metric':eval_metric,
      
}
print(param)


{'learning_rate': (array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 ]),), 'max_depth': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]), 'colsample_bylevel': array([0.3, 0.4, 0.5, 0.6, 0.7]), 'n_estimators': [50, 60, 70, 80], 'eval_metric': 'RMSE'}


In [ ]:
!pip install tpot


     |████████████████████████████████| 92kB 5.8MB/s 
     |████████████████████████████████| 166.7MB 81kB/s 
     |████████████████████████████████| 163kB 50.2MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp37-none-any.whl size=11954 sha256=27ca6b27cb1421a96982a8e7b8bf1bd719cce940e6bd217e37ac5398713d8d5c
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit
  Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
from tpot import TPOTClassifier
from tpot.config import classifier_config_dict 
classifier_config_dict ['catboost.CatBoostClassifier'] = {
'boosting_type': ['gbdt', 'dart'],
'min_child_samples': [1, 5, 7, 10, 15, 20, 35, 50],
'num_leaves': [2, 4, 7, 10, 15, 20, 25, 30, 35, 40, 50], 
'colsample_bytree': [0.7, 0.9, 1.0],
'subsample': [0.7, 0.9, 1.0],
'learning_rate': [0.01, 0.05,0.078, 0.1],
'n_estimators': [5, 20, 35, 50, 75, 100]
}

tpot = TPOTClassifier(generations=5, population_size=24,offspring_size= 12, 
                       verbosity=2, early_stop= 12,
                      config_dict=classifier_config_dict,
                      cv=4,
                      scoring = 'accuracy',)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
tpot.fit(X_train,Y_train)


Generation 1 - Current best internal CV score: 0.7226102941176471

Generation 2 - Current best internal CV score: 0.7226102941176471

Generation 3 - Current best internal CV score: 0.7226102941176471

Generation 4 - Current best internal CV score: 0.7266544117647059

Generation 5 - Current best internal CV score: 0.7266544117647059

Best pipeline: XGBClassifier(input_matrix, learning_rate=0.01, max_depth=8, min_child_weight=14, n_estimators=100, n_jobs=1, subsample=0.25, verbosity=0)


TPOTClassifier(config_dict={'catboost.CatBoostClassifier': {'boosting_type': ['gbdt',
                                                                              'dart'],
                                                            'colsample_bytree': [0.7,
                                                                                 0.9,
                                                                                 1.0],
                                                            'learning_rate': [0.01,
                                                                              0.05,
                                                                              0.1],
                                                            'min_child_samples': [1,
                                                                                  5,
                                                                                  7,
                                                             

In [ ]:
tpot.evaluated_individuals_   #get in detail info about generations, how many kept, how many deropped etc

{'BernoulliNB(FastICA(input_matrix, FastICA__tol=0.5), BernoulliNB__alpha=10.0, BernoulliNB__fit_prior=True)': {'crossover_count': 0,
  'generation': 0,
  'internal_cv_score': 0.7071691176470589,
  'mutation_count': 0,
  'operator_count': 2,
  'predecessor': ('ROOT',)},
 'BernoulliNB(KNeighborsClassifier(input_matrix, KNeighborsClassifier__n_neighbors=21, KNeighborsClassifier__p=2, KNeighborsClassifier__weights=uniform), BernoulliNB__alpha=0.001, BernoulliNB__fit_prior=False)': {'crossover_count': 0,
  'generation': 0,
  'internal_cv_score': 0.6308823529411766,
  'mutation_count': 0,
  'operator_count': 2,
  'predecessor': ('ROOT',)},
 'BernoulliNB(KNeighborsClassifier(input_matrix, KNeighborsClassifier__n_neighbors=42, KNeighborsClassifier__p=1, KNeighborsClassifier__weights=uniform), BernoulliNB__alpha=1.0, BernoulliNB__fit_prior=True)': {'crossover_count': 0,
  'generation': 0,
  'internal_cv_score': 0.6656250000000001,
  'mutation_count': 0,
  'operator_count': 2,
  'predecessor': 

In [ ]:
tpot.get_params

<bound method BaseEstimator.get_params of TPOTClassifier(config_dict={'catboost.CatBoostClassifier': {'boosting_type': ['gbdt',
                                                                              'dart'],
                                                            'colsample_bytree': [0.7,
                                                                                 0.9,
                                                                                 1.0],
                                                            'learning_rate': [0.01,
                                                                              0.05,
                                                                              0.1],
                                                            'min_child_samples': [1,
                                                                                  5,
                                                                                  7,
                   

In [ ]:

accuracy = tpot.score(X_test, Y_test)
print(accuracy)

0.7282352941176471


In [ ]:
from sklearn.metrics import confusion_matrix
ypred = tpot.predict(X_test)

cm = confusion_matrix(Y_test, ypred)
print(cm)

[[ 116  873]
 [  51 2360]]


In [ ]:
from sklearn.metrics import classification_report
cr = classification_report(Y_test, ypred)
print(cr)

              precision    recall  f1-score   support

           0       0.69      0.12      0.20       989
           1       0.73      0.98      0.84      2411

    accuracy                           0.73      3400
   macro avg       0.71      0.55      0.52      3400
weighted avg       0.72      0.73      0.65      3400



In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, tpot.predict_proba(X_test)[:,1])

In [ ]:
from sklearn.metrics import roc_curve, auc,roc_auc_score
roc_auc_score(Y_test, tpot.predict(X_test))

0.5480685717928319